In [11]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('spam.csv', encoding='latin1')  # or encoding='ISO-8859-1'
df.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
990,ham,26th OF JULY,NaN,NaN,NaN
3799,spam,We tried to contact you re your reply to our o...,NaN,NaN,NaN
2213,ham,"Goodmorning, today i am late for 2hrs. Because...",NaN,NaN,NaN
250,ham,Ranjith cal drpd Deeraj and deepak 5min hold,NaN,NaN,NaN
4378,ham,How are you. Just checking up on you,NaN,NaN,NaN


In [18]:
df.shape

(5572, 5)

# Process
1. Data cleaning 
2. EDA
3. Text Preprocessing
4. Model Building 
5. Evaluation
6. Improvements
7. Website
8. Deploy

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


Drop last 3 columns

In [21]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [22]:
df.sample(5)

,v1,v2
5173,ham,U 2.
4768,ham,watever reLation u built up in dis world only ...
3359,ham,Please attend the phone:)
2100,ham,Oh Howda gud gud.. Mathe en samachara chikku:-)
171,ham,"Hmmm.. Thk sure got time to hop ard... Ya, can..."


In [25]:
# rename the columns
df.rename(columns={'v1':'target', 'v2' : 'text'}, inplace=True)
df.sample(5)

,target,text
2777,ham,Kind of. Took it to garage. Centre part of exh...
525,ham,Hi i won't b ard 4 christmas. But do enjoy n m...
407,ham,Hello! Good week? Fancy a drink or something l...
4504,spam,Mobile Club: Choose any of the top quality ite...
2518,ham,Joy's father is John. Then John is the NAME of...


In [28]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['target']= encoder.fit_transform(df['target'])
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [29]:
# missing values
df.isna().sum()

target    0
text      0
dtype: int64

In [33]:
# check for duplicates
df.duplicated().sum()
df = df.drop_duplicates(keep='first')
df.duplicated().sum()

np.int64(0)

In [34]:
df.shape

(5169, 2)